In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import random
import pickle

df=pd.read_csv('/hdd/gros2/zed/src/2_cropped_point_cloud.csv')

n=100 # how many points you want

df1=df.sample(n)

# all the neccessary inputs
torch.manual_seed(1)

# set up the model and the training set 
model = nn.Linear(2,1)
x_coor=df1['x_coor']
y_coor=df1['y_coor']
x1_list=[]
x2_list=[]
x_list=[]

for i in df1.index:
    x1_list.append(x_coor[i])
    x2_list.append(y_coor[i])
    x_list.append([x_coor[i],y_coor[i]])

print(x1_list)

x_train = torch.FloatTensor(x_list)

z_coor=df1['z_coor']
y_list=[]

for i in z_coor:
    y_list.append([i])

y_train = torch.FloatTensor(y_list)

optimizer = torch.optim.SGD(model.parameters(), lr=0.000000001) 

# actual learning process
nb_epochs = 100000
for epoch in range(nb_epochs+1):

    # H(x)
    prediction = model(x_train)

    # finding the cost
    cost = F.mse_loss(prediction, y_train) 

    # reset the gradient
    optimizer.zero_grad()
    cost.backward()
    # update W and b
    optimizer.step()

    # print to show
    if epoch % 10000 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))
        print(list(model.parameters()))

new_var =  torch.FloatTensor([[80,75]]) 
pred_y = model(new_var) 
print("predicted value for [80,75] :", pred_y) 

# data for plotting
w1_final = float(list(model.parameters())[0].data.numpy()[0][0])
w2_final = float(list(model.parameters())[0].data.numpy()[0][1])
b_final = float(list(model.parameters())[1].data.numpy()[0])

y_plot_list=[]
for i in x_train:
    y_plot_list.append(model(i))

fig = plt.figure()
ax = plt.axes(projection='3d')

# 3d lines
xline = np.linspace(0,1,100)
yline = np.linspace(0,1,100)
zline = xline*w1_final + yline*w2_final + b_final
ax.plot3D(xline, yline, zline, 'blue')

x_plot=[]
y_plot=[]
z_plot=[]
s_color = []
for i in range(len(x_list)):
    x_plot.append(x_list[i][0])
    y_plot.append(x_list[i][1])
    z_plot.append(y_list[i])
    s_color.append(1)

# 3d points
s=ax.scatter3D(x_plot, y_plot, z_plot, c="r", s=s_color, alpha=0.05)
s.set_edgecolors = s.set_facecolors = lambda *args:None

ax.set_title("x_coor, y_coor, and z_coor")
ax.set_xlabel("x_coor")
ax.set_ylabel("y_coor")
ax.set_zlabel("z_coor")
plt.show()

ModuleNotFoundError: No module named 'torch'